## Main Function

In [ ]:
list_values = []
IMAGES_FOR_TEST = {}
while True:
    if soil_moisture_activate():
        camera = cv2.VideoCapture(0)
        time.sleep(3)
        take_and_save_photo()
        image_np = loading_image()
        result = inference(image_np, hub_model)
        determine_yell(result)  
        break

In [ ]:
pip install tensorflow protobuf==3.20.3 tensorflow-hub twilio cohere gTTS

In [ ]:
import os
import pathlib
import cv2
import logging
logging.disable(logging.WARNING)

import matplotlib
import matplotlib.pyplot as plt
import time

import numpy as np
from six import BytesIO
from PIL import Image
from six.moves.urllib.request import urlopen

import tensorflow as tf
import tensorflow_hub as hub

import serial

from IPython.display import Audio
from IPython.core.display import display

from twilio.rest import Client
import cohere
from gtts import gTTS

tf.get_logger().setLevel('ERROR')


# Select Model

In [ ]:
model_handle = 'https://tfhub.dev/tensorflow/centernet/hourglass_512x512_kpts/1'
hub_model = hub.load(model_handle)

## Arduino Response

In [ ]:
def soil_moisture_activate():
    arduino = serial.Serial('/dev/cu.usbmodem2101', 9600) # Change com3 when testing
    print('Established serial connection to Arduino')
    arduino_data = arduino.readline()
    
    decoded_value = str(arduino_data[0:len(arduino_data)].decode("utf-8"))
    print(f"Collected data from arduino: {decoded_value}")
    if not decoded_value:
        arduino.close()
        return False
    arduino.close()
    return True

## Webcam to take picture

In [ ]:
def take_and_save_photo():
    ret, frame = camera.read()
    plt.imshow(frame)
    plt.show()
    camera.release()
    cv2.imwrite("photo.png", frame)

    # path to an image
    IMAGES_FOR_TEST["Image1"] = "photo.png"

In [ ]:
def load_image_into_numpy_array(path):
    image = None
    image_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(image_data))
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
    (1, im_height, im_width, 3)).astype(np.uint8)

In [ ]:
def loading_image():
    #@title Image Selection (don't forget to execute the cell!) { display-mode: "form"}
    selected_image = 'Image1' # @param ['Image1']

    image_path = IMAGES_FOR_TEST[selected_image]
    Image.open(image_path)
    image_np = load_image_into_numpy_array(image_path)

    print('min:',np.min(image_np[0]), 'max:', np.max(image_np[0]))
    plt.figure(figsize=(24,32))
    plt.imshow(image_np[0])
    #plt.show()
    return image_np

## Use ML model to make inference

In [ ]:
def inference(image_np, hub_model_fn):
    # running inference
    results = hub_model_fn(image_np)

    # different object detection models have additional results
    # all of them are explained in the documentation
    result = {key:value.numpy() for key,value in results.items()}
    return result

## Determine if yelling is needed

In [ ]:
def send_text():
    client = Client('{clientid}', '{apikey}')
    message = client.messages.create(
        body=generate_prompt(),
        from_='+16264653148',
        to='+12263499481'
    )
    
def generate_prompt():
    co = cohere.Client('{apikey}')
    prompt = "Write a passive agressive message reminding me to water my plants"
    response = co.generate(  
        model='command-nightly',  
        prompt = prompt,  
        max_tokens=200,  
        temperature=0.750
    )
    return response.generations[0].text

def tts():
    mytext = generate_prompt()
    print(mytext)
    language = 'en'
    myobj = gTTS(text=mytext, lang=language, slow=False)
    myobj.save("welcome.mp3")
    os.system("mpg321 welcome.mp3")

def determine_yell(result):
    for i in range(result['detection_classes'][0].size):
        if result['detection_scores'][0][i] > 0.6 and result['detection_classes'][0][i] == 1.0:
            tts()
            return
    send_text()